## Loading the required libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN

## Generating sample dataset

In [ ]:
N = 1000
Tp = 800

t=np.arange(0,N)
x=np.sin(0.02*t)+2*np.random.rand(N)
df= pd.DataFrame(x)
df.head(20)

## Plot them

In [ ]:
plt.plot(df)
plt.show

In [ ]:
#split 'df' dataset into a training and test parts.

values = df.values
train,test = values[0:Tp,:], values[Tp:N,:]
#0 = Start. Tp = 800 For your training. N = 1000
# is to represent step

In [ ]:
'''
RNN model requires a step value that contains n number of elements as an input sequence. Here, we define it as a 'step' x has the following sequece data.
x = [1,2,3,4,5,6,7,8,9,10]
fro step=1, x input and its y prediction become:
x  y
1  2
2  3
3  4
4  5
...
9  10

for step=3, x and y contain:
x      y
1,2,3  4
2,3,4  5
3,4,5  6
4,5,6  7
....
7,8,9  10

As you have noticed the sizes of x input and y output become different. We'll fix it by adding step size into the train'''


In [ ]:
step = 4
#add step elements into train and test
test=np.append(test,np.repeat(test[-1,],step))
train=np.append(train,np.repeat(train[-1,],step))

## convert into dataset matrix

In [ ]:
def convertToMatrix(data, step):
    X,Y = [],[]
    for i in range(len(data)-step):
        d=i+step
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

trainX,trainY = convertToMatrix(train,step)
testX,testY = convertToMatrix(test,step)

In [ ]:
#reshape trainX and testX to fit with Keras model. RNN model requires three-dimensional input data
trainX = np.reshape(trainX,(trainX.shape[0],1,trainX.shape[1]))
testX = np.reshape(testX,(testX.shape[0],1,testX.shape[1]))
trainX.shape
testX.shape

## Building a model with SimpleRNN

In [ ]:
# SimpleRNN model
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(1,step),activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='rmsprop')
model.summary

## Predicting and plotting the result

In [ ]:
#Fit model with trainX data and predict testX data

model.fit(trainX,trainY, epochs=100, batch_size=16, verbose=2)
trainPredict = model.predict(trainX)
testPredict= model.predict(testX)
predicted = np.concatenate((trainPredict,testPredict),axis=0)

## check the loss

In [ ]:
trainScore = model.evaluate(trainX, trainY, verbose=0)
print(trainScore)